In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from sklearn.model_selection import train_test_split

from env_utils import get_data_dir

In [2]:
random_seed = 42

device = torch.cuda.current_accelerator().type if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
def load_image_folder():
    data_directory = get_data_dir()

    transform = transforms.Compose(
        [
            transforms.ToTensor(),
        ]
    )
    image_folder = ImageFolder(f"{data_directory}/pubchem/images", transform)
    return image_folder

In [22]:
image_folder = load_image_folder()
train_data, test_data = train_test_split(image_folder, random_state=random_seed)


batch_size = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Data directory: /Users/Artsem_Nikitsenka/projects/dev-practice/data
Shape of X [N, C, H, W]: torch.Size([3, 3, 100, 100])
Shape of y: torch.Size([3]) torch.int64


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        in_features = 30000
        out_features = 8
        self._linear_relu_stack = nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.ReLU(),
            nn.Linear(out_features, out_features),
            nn.ReLU(),
            nn.Linear(out_features, out_features),
        )

    def forward(self, x):
        x = self.flatten(x)
        print("x=", x)
        print("x.shape=", x.shape)
        logits = self._linear_relu_stack(x)
        return logits

In [8]:
def prepare_model():
    model = NeuralNetwork().to(device)
    print(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    return train_dataloader, test_dataloader, model, loss_fn, optimizer


def train(dataloader: DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss, optimizer: torch.optim.Optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader: DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


def do_training():
    train_dataloader, test_dataloader, model, loss_fn, optimizer = prepare_model()
    print(f"Size of train_dataloader={len(train_dataloader)}")
    print(f"Size of test_dataloader={len(test_dataloader)}")

    epochs = 5
    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
    print("Done!")
    torch.save(model.state_dict(), "img_pytorch_model.pth")
    print("Saved PyTorch Model State to img_pytorch_model.pth")

In [9]:
do_training()

Data directory: /Users/Artsem_Nikitsenka/projects/dev-practice/data
Shape of X [N, C, H, W]: torch.Size([3, 3, 100, 100])
Shape of y: torch.Size([3]) torch.int64
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (_linear_relu_stack): Sequential(
    (0): Linear(in_features=30000, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=8, bias=True)
  )
)
Size of train_dataloader=1
Size of test_dataloader=1
Epoch 1
-------------------------------
x= tensor([[0.2784, 0.2784, 0.2784,  ..., 0.7216, 0.7216, 0.7216],
        [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
        [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
        ...,
        [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
        [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
        [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608]])
x.shape= torch.Size([8, 30000])

In [27]:
print(f"Shape of train_dataloader.dataset.shape: {len(train_dataloader.dataset)}")
print(f"Shape of test_dataloader.dataset.shape: {len(test_dataloader.dataset)}")
test_dataloader.dataset

Shape of train_dataloader.dataset.shape: 8
Shape of test_dataloader.dataset.shape: 3


[(tensor([[[0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           ...,
           [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608]],
  
          [[0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           ...,
           [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608]],
  
          [[0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
           [0.9608, 0.

In [20]:
def load_model():
    model = NeuralNetwork().to(device)
    model.load_state_dict(torch.load("img_pytorch_model.pth", weights_only=True))

    classes = [
        "chem-compound",
        "misc",
    ]

    model.eval()
    x, y = test_dataloader.dataset[0][0], test_dataloader.dataset[0][1]
    with torch.no_grad():
        x = x.to(device)
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f"Predicted: {predicted}, Actual: {actual}")

load_model()

x= tensor([[0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
        [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608],
        [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608]])
x.shape= torch.Size([3, 10000])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x10000 and 30000x8)